# Maintain a file storage of location of GP Surgeries

## Instructions for use
Edit the first cell to configure where the data should be stored.

Based on these parameters, the second cell will:

  - If nothing is downloaded, download the addresses of all GP surgeries in the UK (quick), then use the
    Nominatim API to fetch the lat/lon location. This is rate-limited to 1 qps, so for the roughly 16,000
    surgeries takes about 4-5 hours.
  - If the data already exists, do nothing
  - If the GP address data is downloaded, but the lat/lon has not finished populating, it will resume populating
    resaving the output every minute
  - If the previous_csv_file_path is not empty, then it will be used for address lookups in preference to 
    Nominatim. This should mean when the data updates, only changes should need to be looked up, and the run time
    should be much quicker, depending on the number of changes
    
So if you want to prompt an update:

  - move the data from the path in `target_csv_file_path` to a new location
  - put the new location in `previous_csv_file_path`
  - potentially update the header list if the headers have changed for the CSV
  - run the cell
  
    

In [34]:
# This could be done with papermill parameters. Instead, each parameter is a single variable 
# assignment with a comment

# URL to fetch the most recent EPRACCUR dataset from NHS Digital  webpage
epraccur_source_url = 'https://files.digital.nhs.uk/assets/ods/current/epraccur.zip'

# Output location for the final CSV file with the Epraccur data, unzipped, with lat, long information
target_csv_file_path = 'data/gp_data.csv'

# If this is an incremental update, location for the CSV file of previous output
# If this is a first run, or a complete refresh is needed, set to None
previous_csv_file_path = None # 'data/gp_data_old.csv'

# Column headers for the EPRACCUR data (this is not in the CSV file)
epraccur_headers = ["Organisation Code", "Name", "National Grouping", "High Level Health Geography", "Address line 1", "Address line 2", "Address line 3", 
"Address line 4", "Address line 5","Postcode","Open Date","Close Date","Status Code","Organisation Sub-Type Code","Commissioner","Join Provider/Purchaser Date",
"Left Provider/Purchaser Date","Contact Telephone Number", "Null 1", "Null 2", "Null 3", "Amended Record Indicator", "Null 4", "Provider/Purchaser",
"Null 5", "Prescribing Setting", "Null 6"]

In [35]:
import csv
import io
import numpy as np
import os
import pandas as pd
import urllib.request
import zipfile
from geopy.geocoders import Nominatim, Photon
from geopy.extra.rate_limiter import RateLimiter

In [36]:
# EPRACCUR GP Practice data ingestion
#-----------------------------------------

# Retrieves and saves the most recent EPRACCUR dataset from NHS Digital webpage
# Adds in headers
#-----------------------------------------------------------------------------

if not os.path.exists(target_csv_file_path):
    filehandle, _ = urllib.request.urlretrieve(epraccur_source_url)
    with zipfile.ZipFile(filehandle, 'r') as zip_file_object:
        first_file = zip_file_object.namelist()[0]
        with zip_file_object.open(first_file) as source_file:
            reader = csv.reader(io.TextIOWrapper(source_file, 'utf-8'))
            with open(target_csv_file_path, 'w') as csv_file:
                writer = csv.writer(csv_file)
                writer.writerow(epraccur_headers) # write the headers into the CSV
                for row in reader: # copy the content
                    writer.writerow(row)


In [37]:
# Define the methods for getting the latitude and longitude from a data row.
# Includes which geocoder to use, and what combinations of addresses to try
# -------------------------------------------------------------------------

geolocator = Nominatim(user_agent="open_access_nhs_2") #------------- Setting Nominatim as the API to use during postcode geolocation
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=3) #--------- Rate limiting to 1 query per second
#geolocator = Photon(user_agent="open_access_nhs") #------------- Setting Photon as the API to use during postcode geolocation
#geocode = RateLimiter(geolocator.geocode, min_delay_seconds=2) #--------- Rate limiting to 1 query per second

def is_england(location):
    if location is None:
        return False
    england = location.latitude > 50 and location.latitude < 56 and location.longitude > -6 and location.longitude < 2
    if not england:
        print("location {},{} not england".format(location.latitude, location.longitude))
    return england

def is_uk(location):
    if location is None:
        return False
    # Go south to the channel islands, and north and west to scotland
    uk = location.latitude > 49 and location.latitude < 61 and location.longitude > -9 and location.longitude < 2
    if not uk:
        print("location {},{} not england".format(location.latitude, location.longitude))
    return uk
    
def address_lat_and_lon(row):
    postcode = '' if pd.isnull(row['Postcode']) else str(row['Postcode'])
    if postcode.lower() == 'nan':
        postcode = ''
    # concatenate the address into something we can use with                    
    address = ', '.join([str(row['Address line 2']).title(), str(row['Address line 3']).title(), str(row['Address line 4']).title(), postcode])
    address = address.replace('Nan', '').replace(' ,', '')

    # If BFPO is overseas, skip it
    if str(row['Address line 4']).title() == 'Bfpo':
        print("skipping BFPO")
        return (address, None, None)
    
    print("trying {}".format(address))
    location = geocode(address)
    
    # if it fails, try without line 4
    if not is_uk(location):
        address2 = ', '.join([str(row['Address line 2']).title(), str(row['Address line 3']).title(), postcode])
        address2 = address2.replace('Nan', '').replace(' ,', '')
        print("trying {}".format(address2))
        location = geocode(address2)
        if location is not None:
            address = address2
    # If this fails, just use postcode
    if not is_uk(location) and postcode != "":
        print("trying {}".format(postcode))
        location = geocode(postcode)
        if location is not None:
            address = postcode
        else:
            print("Failed postcode only: " + postcode)
    
    if not is_uk(location):
        return (address, None, None)
    return (address, location.latitude, location.longitude)



In [38]:
# Define the methods for handling caches

# Cache by postcode, by both the previous file if set, and the current file
def add_to_latlong_cache(df, cache):
    if ('latitude' not in df.columns) or ('longitude' not in df.columns) or ('Postcode' not in df.columns):
        return
    for index, row in df.iterrows():
        if 'latitude' not in row:
            continue
        if pd.isnull(row['latitude']):
            continue
        if 'longitude' not in row:
            continue
        if pd.isnull(row['longitude']):
            continue
        if 'Postcode' not in row:
            continue
        if pd.isnull(row['Postcode']):
            continue
        # in case of errors, make sure we never add 'nan' or 'Nan' as key
        if str(row['Postcode']).lower() == 'nan':
            return
        cache[row['Postcode']] = (row['latitude'], row['longitude'])



In [39]:
# Update the lat and long columns, using a cache to minimize requests
# -------------------------------------------------------------------

postcode_to_latlong = {}

if previous_csv_file_path is not None and os.path.exists(previous_csv_file_path):
    print("updating cache from previous file")
    previous_practice_df = pd.read_csv(previous_csv_file_path)
    add_to_latlong_cache(previous_practice_df, postcode_to_latlong)
    print("done")

gp_practice_df = pd.read_csv(target_csv_file_path)

print("updating cache from existing file")
add_to_latlong_cache(gp_practice_df, postcode_to_latlong)
print("done")

# Add empty latitude and longitude columns if they don't exist (we have a new file)
if not 'latitude' in gp_practice_df.columns:
    gp_practice_df['latitude'] = np.nan
if not 'longitude' in gp_practice_df.columns:
    gp_practice_df['longitude'] = np.nan


# Look up lat and long a row at a time, saving every minute in case of errors.
# Iterating a row at a time isn't very "pandas" style. But we want to save every minute (roughly 60 rows).
changed = False
change_count = 0
for index, row in gp_practice_df[gp_practice_df['latitude'].isnull()].iterrows():
    if change_count % 20 == 0 and changed:
        changed = False
        print("Updating saved file")
        gp_practice_df.to_csv(target_csv_file_path)
        print("Done")
    if str(row['Postcode']).lower() == 'nan':
        # Never lookup 'nan' - it can be found!
        continue
    if row['Postcode'] in postcode_to_latlong:
        lat, long = postcode_to_latlong[row['Postcode']]
        print("Found in cache: " + str(row['Postcode']))
        changed = True
        change_count +=1
        gp_practice_df.at[index, 'latitude'] = lat
        gp_practice_df.at[index, 'longitude'] = long
        continue
    print("Looking up " + row['Name'])
    address, lat, long = address_lat_and_lon(row)
    if lat is None:
        print('Failed for ' + address)
        continue
    else:
        print('Found')
    gp_practice_df.at[index, 'Full address'] = address
    gp_practice_df.at[index, 'latitude'] = lat
    gp_practice_df.at[index, 'longitude'] = long
    postcode_to_latlong[row['Postcode']] = (lat, long)
    change_count +=1
    changed = True
gp_practice_df.to_csv(target_csv_file_path)

print('{} rows updated'.format(change_count))

updating cache from previous file
done
updating cache from existing file
done
Found in cache: TS18 1HU
Found in cache: TS18 2AW
Found in cache: TS26 8DB
Found in cache: TS5 8SB
Found in cache: TS14 7DJ
Found in cache: TS18 2AT
Found in cache: TS24 7PW
Found in cache: TS6 6TD
Found in cache: TS5 6HF
Found in cache: TS24 7PW
Found in cache: TS3 6AL
Found in cache: TS12 2FF
Found in cache: TS23 2LA
Found in cache: TS10 1TZ
Found in cache: TS1 3QY
Found in cache: TS17 0EE
Found in cache: TS10 4NW
Found in cache: TS3 7RL
Found in cache: TS4 3BU
Found in cache: TS6 6TD
Updating saved file
Done
Found in cache: TS12 2TG
Found in cache: TS1 2NX
Found in cache: TS18 1HU
Found in cache: TS5 6HA
Found in cache: TS15 9DD
Found in cache: TS1 2NX
Found in cache: TS1 3RY
Found in cache: TS24 7PW
Found in cache: TS14 7DJ
Found in cache: TS3 6AL
Found in cache: TS17 0EE
Found in cache: TS1 3RX
Found in cache: TS20 2UZ
Found in cache: TS1 2NX
Found in cache: TS3 6AL
Found in cache: TS16 9EA
Found in cach

Done
Found in cache: NE4 8QB
Found in cache: NE5 1LJ
Found in cache: NE6 4QD
Found in cache: NE6 1SA
Found in cache: NE5 2LH
Found in cache: NE15 9PA
Found in cache: NE1 7XR
Found in cache: NE3 1DQ
Found in cache: NE6 1SG
Found in cache: NE15 6TQ
Found in cache: NE4 9XD
Found in cache: NE3 5NF
Found in cache: NE3 5JP
Found in cache: NE3 1TX
Found in cache: NE4 6SH
Found in cache: NE15 8LX
Found in cache: NE6 2NN
Found in cache: NE7 7TD
Found in cache: NE5 2UY
Found in cache: NE29 0HQ
Updating saved file
Done
Found in cache: NE28 6RZ
Found in cache: NE29 0SF
Found in cache: NE26 2ND
Found in cache: NE26 1NA
Found in cache: NE12 9BQ
Found in cache: NE26 3LW
Found in cache: NE29 0HT
Found in cache: NE25 9XJ
Found in cache: NE13 6LN
Found in cache: NE28 0AD
Found in cache: NE27 0HJ
Found in cache: NE29 0SF
Found in cache: NE28 6BB
Found in cache: NE13 6JJ
Found in cache: NE29 0DZ
Found in cache: NE25 9PH
Found in cache: NE27 0HJ
Found in cache: NE27 0HJ
Found in cache: NE28 0PP
Found in ca

Done
Found in cache: DH3 3JX
Found in cache: LN6 9PT
Found in cache: B30 2XR
Found in cache: YO31 8TA
Found in cache: YO17 7NG
Found in cache: YO12 6DN
Found in cache: RG19 8ET
Found in cache: DA2 7AF
Found in cache: SE18 3RG
Found in cache: SE18 3RG
Found in cache: RG45 7EG
Found in cache: L31 1HW
Found in cache: NG3 6AA
Found in cache: RM7 0AG
Found in cache: SL2 4HL
Found in cache: NE1 4LP
Found in cache: HA1 3UJ
Found in cache: PO19 6SE
Found in cache: LE3 9QP
Found in cache: OX16 9AL
Updating saved file
Done
Found in cache: LN2 5QY
Found in cache: HX3 0PW
Found in cache: BS2 8HW
Found in cache: BL9 7TD
Found in cache: B71 4HJ
Found in cache: HP2 4AD
Found in cache: NG5 1PB
Found in cache: DL14 6AD
Found in cache: W6 8RF
Found in cache: NW1 2BU
Found in cache: B75 7RR
Found in cache: TS19 8PE
Found in cache: SE5 9RS
Found in cache: S44 5BL
Found in cache: L7 8XP
Found in cache: M13 9WL
Found in cache: TR1 3LJ
Found in cache: MK6 5LD
Found in cache: SY3 8XQ
Found in cache: WR5 1DD
U

trying 28 Gillygate, York, YO31 7WQ
trying YO31 7WQ
Failed postcode only: YO31 7WQ
Failed for 28 Gillygate, York, YO31 7WQ
Found in cache: YO24 4HD
Found in cache: YO21 2PA
Found in cache: YO12 7AS
Found in cache: HG1 5AR
Found in cache: YO11 1UB
Found in cache: YO26 5LD
Found in cache: YO8 4BL
Found in cache: YO1 7NP
Found in cache: BD23 5LS
Found in cache: YO24 3BZ
Found in cache: YO22 5DP
Found in cache: YO31 7YD
Found in cache: DL9 3JD
Found in cache: LS24 8HD
Found in cache: YO12 5SD
Found in cache: YO62 4EF
Found in cache: YO12 7RE
Found in cache: YO60 6PS
Found in cache: DL11 6SU
Updating saved file
Done
Found in cache: HG3 2XT
Found in cache: YO14 0NN
Found in cache: YO31 1EX
Found in cache: YO8 4ET
Found in cache: YO11 1XE
Found in cache: DL7 8LZ
Found in cache: DL9 3JD
Found in cache: YO31 7WA
Found in cache: YO11 1XE
Found in cache: LS29 8TQ
Found in cache: BD3 7HZ
Found in cache: BD20 0DG
Found in cache: BD9 6LL
Found in cache: BD21 2JH
Found in cache: BD12 9NG
Found in cac

Done
Found in cache: LS14 2EH
Found in cache: LS12 5SG
Found in cache: LS23 6RN
Found in cache: LS7 3QB
Found in cache: LS9 7TA
Found in cache: BD11 1JU
Found in cache: LS22 6JS
Found in cache: LS10 4HT
Found in cache: LS10 4UZ
Found in cache: LS17 6AW
Found in cache: WF4 4PA
Found in cache: WF4 5BY
Found in cache: WF7 7HZ
Found in cache: WF1 4PR
Found in cache: WF10 1PH
Found in cache: WF9 3AP
Found in cache: WF8 1NF
Found in cache: WF2 8FE
Found in cache: WF7 5HE
Found in cache: WF8 1SU
Updating saved file
Done
Found in cache: WF1 5PN
Found in cache: WF1 2PE
Found in cache: WF8 4PQ
Found in cache: WF9 2RD
Found in cache: WF1 1PG
Found in cache: WF10 4EN
Found in cache: WF3 4HS
Found in cache: WF2 7GP
Found in cache: WF11 0JA
Found in cache: WF2 9PE
Found in cache: WF10 3TG
Found in cache: WF10 1DP
Found in cache: WF9 4DP
Found in cache: WF1 3JQ
Found in cache: WF4 1HJ
Found in cache: WF1 1PG
Found in cache: LS26 9AA
Found in cache: WF5 8DF
Found in cache: WF7 6JL
Found in cache: WF6 

...

Done
Found in cache: DH1 2QW
Found in cache: OL1 1NL
Found in cache: DH8 0BW
Found in cache: GU9 9QL
Found in cache: B20 2NR
Found in cache: DH7 6JW
Found in cache: SW6 4UL
Found in cache: SW19 5NJ
Found in cache: SM4 5DX
Found in cache: ST6 7AG
Found in cache: B69 1HE
Found in cache: B70 0PU
Found in cache: NW1 0PE
Found in cache: WS2 9XH
Found in cache: SW15 3SW
Found in cache: SW11 2LN
Found in cache: L35 5DR
Found in cache: OL6 6NG
Found in cache: SO30 4FQ
Found in cache: L35 2YZ
Updating saved file
Done
Found in cache: LS1 3EX
Found in cache: LS9 7TF
Found in cache: B19 2JA
Found in cache: M30 0BL
Found in cache: SE1 3ER
Found in cache: M6 8HD
Found in cache: B70 7AW
Found in cache: PE21 9QS
Found in cache: LN2 5QY
Found in cache: WA2 8WA
Found in cache: NE45 5LG
Found in cache: CM23 5JH
Found in cache: BD1 3ND
Found in cache: CV1 4AE
Found in cache: AL6 9PW
Found in cache: MK40 4AW
Found in cache: GL16 8RH
Found in cache: RH4 2AA
Found in cache: E1 4AH
Found in cache: KT19 8PB
Up

Done
Found in cache: HR1 2BG
Found in cache: WN1 2NN
Found in cache: WN4 0XD
Found in cache: DH2 3DJ
Found in cache: CB2 1LR
Found in cache: PE27 5JD
Found in cache: ST17 9AQ
Found in cache: LS16 6FR
Found in cache: CM2 5PF
Found in cache: B75 6DX
Found in cache: B35 6EE
Found in cache: B23 6DJ
Found in cache: B73 6SP
Found in cache: B44 0PU
Found in cache: M34 2GP
Found in cache: S21 4BZ
Found in cache: DH4 5GU
Found in cache: DN15 6EA
Found in cache: S42 6PX
Found in cache: DN16 2SZ
Updating saved file
Done
Found in cache: E8 4SA
Found in cache: HP6 6FA
Found in cache: CT14 9UA
Found in cache: ME2 4UW
Found in cache: HP6 6FA
Found in cache: HP6 6FA
Found in cache: MK42 9DJ
Found in cache: NN16 8DN
Found in cache: CR7 7JH
Found in cache: PR5 5XU
Found in cache: PR5 5XU
Found in cache: CR8 2YL
Found in cache: L13 1FB
Found in cache: DN21 2UQ
Found in cache: LN2 3JH
Found in cache: LN6 9AY
Found in cache: LN4 3EZ
Found in cache: L13 1FB
Found in cache: CV32 5QB
Found in cache: KT16 0PZ


Done
Found in cache: M41 5SL
Found in cache: B9 4EY
Found in cache: S43 4FR
Found in cache: WN1 2NN
Found in cache: HD6 3NA
Found in cache: HX2 9LL
Found in cache: HX6 3AB
Found in cache: OL14 5RN
Found in cache: SK23 0RG
Found in cache: LE10 2SE
Found in cache: LE67 4DR
Found in cache: LE18 4SE
Found in cache: LE2 0TA
Found in cache: IP4 2LA
Found in cache: BN1 6AG
Found in cache: ST3 1SQ
Found in cache: WV3 0JH
Found in cache: WA5 1QG
Found in cache: LS13 2BL
Found in cache: B92 7SA
Updating saved file
Done
Found in cache: WF12 9QU
Found in cache: NG4 1JA
Found in cache: UB2 4SD
Found in cache: B36 9EJ
Found in cache: CV7 7RW
Found in cache: B37 7WB
Found in cache: B90 4DD
Found in cache: CR7 7JH
Found in cache: SO40 2RZ
Found in cache: PO17 5NA
Found in cache: SO40 2RZ
Found in cache: LS29 8TH
Found in cache: GL51 9EZ
Found in cache: GL51 9EZ
Found in cache: SR4 7TP
Found in cache: SE1 7EU
Found in cache: S61 1AJ
Found in cache: ME8 0PZ
Found in cache: DN15 8QZ
Found in cache: RG1 7

Done
Found in cache: DN4 8QN
Found in cache: BN2 4SE
Found in cache: BN2 4SE
Found in cache: DN1 1JE
Found in cache: RH11 7DH
Found in cache: DN4 6EN
Found in cache: MK40 4HR
Found in cache: MK14 6GL
Found in cache: M4 6LN
Found in cache: WA15 0NU
Found in cache: DL3 7JX
Found in cache: E13 8SL
Found in cache: IG11 9LX
Found in cache: B10 0AU
Found in cache: RH4 2AA
Found in cache: NE23 6US
Found in cache: SS17 7QQ
Found in cache: LU4 0TR
Found in cache: SS17 7QQ
Found in cache: ST18 0XF
Updating saved file
Done
Found in cache: SG14 1JA
Found in cache: BD21 4AD
Found in cache: L7 8XP
Found in cache: DN5 0AT
Found in cache: DN1 1TJ
Found in cache: SS14 1EH
Found in cache: PL9 9XA
Found in cache: KT18 7EG
Found in cache: S61 1AJ
Found in cache: HP7 0JD
Found in cache: TN24 0LZ
Found in cache: WN1 2NN
Found in cache: WN7 1HS
Found in cache: MK2 2EH
Found in cache: GL51 9EZ
Found in cache: LS15 8GB
Found in cache: BS24 9AY
Found in cache: E8 3SG
Found in cache: E8 3SG
Found in cache: SE1 0

Done
Found in cache: DY4 0SN
Found in cache: SE1 7EH
Found in cache: CR7 7JH
Found in cache: HD9 3TR
Found in cache: HD2 2RU
Found in cache: HD1 3AL
Found in cache: SL6 6DU
Found in cache: BN2 3EW
Found in cache: SS15 5HL
Found in cache: S60 2UD
Found in cache: B19 2TJ
Found in cache: CM16 6TN
Found in cache: B66 4BJ
Found in cache: SW19 1RQ
Found in cache: NW8 0AD
Found in cache: SS4 1RB
Found in cache: LN5 7JH
Found in cache: CV1 4JH
Found in cache: MK16 8QT
Found in cache: SG18 8DJ
Updating saved file
Done
Found in cache: DE1 2QY
Found in cache: DE1 2QY
Found in cache: BD9 6DP
Found in cache: DN4 8QN
Found in cache: BR3 3FD
Found in cache: CM15 8DR
Found in cache: CV1 4JU
Found in cache: M33 7XF
Found in cache: M33 5PN
Found in cache: M41 9JN
Found in cache: M41 0TS
Found in cache: WA14 1RN
Found in cache: M33 7ZZ
Found in cache: M41 0TZ
Found in cache: DN4 8QN
Found in cache: DN4 6ED
Found in cache: DN4 6ED
Found in cache: DN4 8QN
Found in cache: DN4 8QN
Found in cache: DN4 8JS
Upd

In [40]:
# Update status
df = pd.read_csv(target_csv_file_path)
print('{total} rows, {lat} with latitude'.format(total=len(df.index), lat=df['latitude'].count()))

15199 rows, 14991 with latitude


In [137]:
# Clears out all stored Latitude and Longitude
# execute with care

#df = pd.read_csv(target_csv_file_path)
#df.loc[pd.isnull(df['Postcode']), 'latitude'] = np.nan
#df.loc[pd.isnull(df['Postcode']), 'longitude'] = np.nan
#df.to_csv(target_csv_file_path)
